# Nursery Seed-Store Widget

In [1]:
# Load libraries
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
# Load data
hist = pd.read_csv("../data/accession-items-history_2017to2025.csv", low_memory=False)

# Filter current seed-store items
seed_store = hist.loc[
    (hist["Current"] == ">>>") &
    (hist["ItemStatus"] == "Seed store"),
    ["AccNoFull","ItemSpecCount","TaxonName","Purpose"]
].copy()

# Summarize inventory
inventory = (
    seed_store
    .groupby(["AccNoFull","TaxonName","Purpose"], as_index=False)
    .agg(SeedsInStore=("ItemSpecCount","sum"))
    # reorder columns
    [["AccNoFull","TaxonName","Purpose","SeedsInStore"]]
)

# 4) Build interactive table
acc_sel       = widgets.Text(description="Accession #:", placeholder="e.g. 1978-045")
taxon_sel     = widgets.Text(description="Taxon:", placeholder="any substring")
purpose_sel   = widgets.Dropdown(
    options=["All"] + sorted(inventory["Purpose"].dropna().unique().tolist()),
    description="Purpose:"
)
output        = widgets.Output()

def refresh(_=None):
    output.clear_output()
    df = inventory.copy()
    
    # apply filters
    if acc_sel.value.strip():
        df = df[df["AccNoFull"].str.contains(acc_sel.value.strip(), case=False)]
    if taxon_sel.value.strip():
        df = df[df["TaxonName"].str.lower().str.contains(taxon_sel.value.strip().lower())]
    if purpose_sel.value != "All":
        df = df[df["Purpose"] == purpose_sel.value]
    
    with output:
        display(df)

# wire up callbacks
for w in (acc_sel, taxon_sel, purpose_sel):
    w.observe(refresh, names="value")

# initial display
controls = widgets.HBox([acc_sel, taxon_sel, purpose_sel])
display(controls, output)
refresh()

Output()